# Identifying gene regulation modules for tumor metastasis through combinatorial fusion analysis
**Description: We developed an efficient bioinformatics approach to identify metastasis-associated
gene regulatory modules (GRMs) in cancer networks. Using a subgraph method to extract
GRMs, we applied combinatorial fusion analysis (CFA) to prioritize them based on relevance.** \
There are 7 main modules:
- Module01: Calculate the LogFC and Hazard ratio
- Module02: Count the occurences of KIRC genes in three databases
- Module03: Cancer driver gene (COSMIS database)
- Module04: CFA calculation Avg and Weigthed Score Combination (using 11 combinations of the features)
- Module05: Find TOP2 and BOTTOM2 from Weigthed Score Combination result
- Module06: Calculate frequency item-ID TOP5 and BOTTOM5. It is chosen from the highest and lowest five ranked items-Id for each of the 11 combinations.
- Module07: Calculate Jaccard index to Quantify the difference between the weighted SC and average SC 



In [ ]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
from itertools import islice


In [ ]:
pwd

## Subgraph Dataset:

In [ ]:
#from google.colab import drive

# Define the working directory automatically
#drive.mount('/content/drive')

# Path to the directory containing all the pattern of three node subgraphs
subgraph = "/Users/anindaastuti/CFA Master/4node"
wkdr = "/Users/anindaastuti/"
print(subgraph)
print(wkdr)


# MODULE01: CALCULATE LogFC AND HAZARD RATIO: KIRC
**Read the TMMGdb:** \
In order to get the full information from TMMGdb, the filtering conditions as follows: 

If |LogFC|< 0.38  
	Then  LogFC_label = 0
Else if |LogFC| > 0.46   
	Then   LogFC_label = 10
Else if |LogFC | >0.38 & <= 0.46
	Then   LogFC_label = Label

If  HR < 1.05  
	Then  HR_label = 0
Else if  HR > 1.20  
	Then   LogFC_label = 10
Else if  HR >1.05 & <= 1.20   
	Then   LogFC_label = Label

 
Total 25975 genes were collected.

### STEP01: HANDLE THE TMMGdb:

In [ ]:
#Read the TMMGdb-KIRC
KIRC = pd.read_excel(f"{wkdr}/data-tmmgdb-logFC-HR-pvalue-BRCA.xlsx")
print(KIRC.columns)


#Rename columns
KIRC.rename(columns={"Adj-Pvalue.1":"Pvalue_HR", "Adj-Pvalue":"Pvalue_Log(FC)"},inplace=True)
print(KIRC.shape)

#Create 2 Series: pvalue_series and HR_series
mapping_pvalue_logFC  = dict(zip(KIRC['Gene'], KIRC['Pvalue_Log(FC)']))
mapping_pvalue_HR     = dict(zip(KIRC['Gene'], KIRC['Pvalue_HR']))
mapping_HR            = dict(zip(KIRC['Gene'], KIRC['HR']))
mapping_logFC         = dict(zip(KIRC['Gene'], KIRC['Log(FC)']))

print(f"mapping_pvalue: {len(mapping_pvalue_logFC)}")
print(f"mapping_Pvalue_HR: {len(mapping_pvalue_HR)}")
print(f"mapping_HR: {len(mapping_HR)}")


In [ ]:
#abs means absolute
#logFC_max_score = np.abs(KIRC['Log(FC)']).max()
#logFC_min_score = np.abs(KIRC['Log(FC)']).min()
logFC_min_score=0.38
logFC_max_score=0.46
print("logFC_max_score",logFC_max_score)
print("logFC_min_score",logFC_min_score)
# Calculate the range size
n_range=10
range_size = (logFC_max_score - logFC_min_score) / n_range
print("range_size",range_size)
# Generate the range values
#logFC_min_score + i * range_size+0.000001 digunakan untuk membuat batas bawah
#ketika index i =0 maka min score adalah logFC_min_score
#if i == 0 digunakan untuk nilai awal adalah 0.38 dan selanjutnya di tambah 0.000001
#logFC_min_score + (i + 1) * range_size digunakan untuk membuat batas atas
#ketika index i =0 maka max score adalah logFC_min_score+range_size
ranges = [(logFC_min_score + i * range_size if i == 0 else logFC_min_score + i * range_size + 0.001, logFC_min_score + (i + 1) * range_size) for i in range(n_range)]

# Labels for the ranges
labels = [i+1 for i in range(n_range)]

# Create a list of tuples containing the range labels, maximum and minimum values
#zip digunakan untuk menggabungkan 2 list data
#(label, r[0], r[1]) digunakan untuk membuat setiap baris data, label: 1-5, r[0]: batas bawah, r[1]: batas atas
#for label, r in zip(labels, ranges) digunakan untuk mengekstrak data label dan juga ranges
# r adalah data ranges
range_data = [(label, r[0], r[1]) for label, r in zip(labels, ranges)]

# Create a DataFrame from the list of tuples
range_df = pd.DataFrame(range_data, columns=['LogFC_Label', 'LogFC_Min', 'LogFC_Max'])

print(range_df)

In [ ]:
#HR_max_score = KIRC['HR'].max()
#HR_min_score = KIRC['HR'].min()
HR_max_score=1.20
HR_min_score=1.05
print("HR_max_score",HR_max_score)
print("HR_min_score",HR_min_score)
# Calculate the range size
n_range_HR=10
HR_range_size = (HR_max_score - HR_min_score) / n_range_HR
print("HR_range_size",HR_range_size)
# Generate the range values
HR_ranges = [(HR_min_score + i * HR_range_size if i == 0 else HR_min_score + i * HR_range_size+0.001, HR_min_score + (i + 1) * HR_range_size) for i in range(n_range_HR)]

# Labels for the ranges
HR_labels = [i+1 for i in range(n_range_HR)]

# Create a list of tuples containing the range labels, maximum and minimum values
HR_range_data = [(label, r[0], r[1]) for label, r in zip(HR_labels, HR_ranges)]

# Create a DataFrame from the list of tuples
HR_range_df = pd.DataFrame(HR_range_data, columns=['HR_Label', 'HR_Min', 'HR_Max'])

print(HR_range_df)

### Step 02: Calculate P-value and Hazard ratio of each node

**Process through each 4-subgraph network type**

In [ ]:
import pandas as pd
import os
import glob

# Assuming you have defined mapping_pvalue, mapping_LogFC, and mapping_HR functions


#folder_path = "/Users/anindaastuti/CFA Master Code-breast cancer [BRCA]/4node"


default_value_for_logFC=0
default_value_for_HR=0
# Create a new Excel workbook
print("cek data")
outputfolder = f"{wkdr}/output files"
output_file_path = f"{outputfolder}/result_Pvalue-HR.xlsx"

# Get a list of all files in the folder
all_files = [os.path.join(subgraph, f) for f in os.listdir(subgraph) if os.path.isfile(os.path.join(subgraph, f))]

with pd.ExcelWriter(output_file_path) as writer:
    for subgraph_file in all_files:
        print(f"Reading file: {subgraph_file}")
        with open(subgraph_file, "r", encoding="utf-8", errors="ignore") as file:
            print(file.read())
        
        subgraph_name = os.path.basename(subgraph_file).split(".")[0]
        print(subgraph_name)
        subgraph_data = pd.read_csv(subgraph_file, sep="\t", skiprows=1, header=None)
        
        # Removing the last row
        subgraph_data = subgraph_data.drop(subgraph_data.index[-1])

        # Removing whitespaces in dataframe
        subgraph_data = subgraph_data.replace('\\s+', '', regex=True)

        # Split the original column into 3 columns named: gene1, gene2, gene3
        subgraph_data[["gene1", "gene2", "gene3","gene4"]] = subgraph_data[0].str.split("|", expand=True)
        subgraph_data["0"]=subgraph_data[0]
        # Adding Pvalue and HR of each gene into the dataframe
        for gene in ["gene1", "gene2", "gene3","gene4"]:
            subgraph_data["logFC:" + gene] = subgraph_data[gene].map(mapping_logFC)
            subgraph_data["logFC_adj_p:" + gene] = subgraph_data[gene].map(mapping_pvalue_logFC)
            subgraph_data["HR:" + gene] = subgraph_data[gene].map(mapping_HR)
            subgraph_data["HR_adj_p:" + gene] = subgraph_data[gene].map(mapping_pvalue_HR)

        # Calculate the compounded P-value and compounded HR if needed
        # ...

        # Other calculations


# Assuming 'gene' is a variable containing the gene name
            # abs means absolute value
            subgraph_data["logFC_adj_p<0.05:" + gene] = np.where((subgraph_data["logFC_adj_p:" + gene] < 0.05), 1, 0)

            subgraph_data["HR_adj_p<0.05:" + gene] = np.where((subgraph_data["HR_adj_p:" + gene] < 0.05), 1, 0)


            # Create empty lists to store the range labels
            logFC_range_labels = []
            HR_range_labels = []

            # LogFC Calculation
            # Iterate over each row of subgraph_data
            for index, row in subgraph_data.iterrows():
                logFC_value = np.abs(row["logFC:" + gene])
                HR_value = row["HR:" + gene]
                 # Take only 4 significant figures
                logFC_value = float(f"{logFC_value:.4g}")

                HR_value = float(f"{HR_value:.4g}")
    # Check which range label the logFC_value belongs to
                logFC_label_found = False
                if logFC_value < logFC_min_score:
                    logFC_range_labels.append(0)
                    logFC_label_found = True
                elif logFC_value >logFC_max_score:
                    logFC_range_labels.append(10)
                    logFC_label_found = True
                else:
                    for _, range_row in range_df.iterrows():
                        if range_row["LogFC_Min"] <= logFC_value <= range_row["LogFC_Max"]:
                            logFC_range_labels.append(range_row["LogFC_Label"])
                            logFC_label_found = True
                            print("logFC_check:",logFC_value)
                            print("logFC_range_labels:",logFC_range_labels)
                            #break

  # Check if a label was found, if not, append a default value
                if not logFC_label_found:
                    logFC_range_labels.append(default_value_for_logFC)

                # Check which range label the HR_value belongs to
                HR_label_found = False
                if HR_value < HR_min_score:
                    HR_range_labels.append(0)
                    HR_label_found = True
                elif HR_value > HR_max_score:
                    HR_range_labels.append(10)
                    HR_label_found = True
                else:
                    for _, HR_range_row in HR_range_df.iterrows():
                        if HR_range_row["HR_Min"] <= HR_value <= HR_range_row["HR_Max"]:
                            HR_range_labels.append(HR_range_row["HR_Label"])
                            HR_label_found = True
                            #break

              # Check if a label was found, if not, append a default value
                if not HR_label_found:
                  HR_range_labels.append(default_value_for_HR)

# After the loop, assign the lists to DataFrame columns
            subgraph_data["logFC_Label:" + gene] = logFC_range_labels
            print("result LogFC Label:",subgraph_data["logFC_Label:" + gene])
            subgraph_data["HR_Label:" + gene] = HR_range_labels
        subgraph_data['Total_Combination_logFC'] = subgraph_data[['logFC_Label:gene1','logFC_adj_p<0.05:gene1','logFC_Label:gene2','logFC_adj_p<0.05:gene2','logFC_Label:gene3','logFC_adj_p<0.05:gene3','logFC_Label:gene4','logFC_adj_p<0.05:gene4']].astype(int).sum(axis=1)
        subgraph_data['Total_Combination_HR'] = subgraph_data[['HR_Label:gene1','HR_adj_p<0.05:gene1','HR_Label:gene2','HR_adj_p<0.05:gene2','HR_Label:gene3','HR_adj_p<0.05:gene3','HR_Label:gene4','HR_adj_p<0.05:gene4']].astype(int).sum(axis=1)


        # Rearrange the columns in subgraph_data as per your preference
        subgraph_data = subgraph_data[[
            # Place the columns you want in the desired order
            '0',
            'gene1', 'gene2', 'gene3','gene4',
            # Other columns...
            'logFC:gene1','logFC_adj_p:gene1','HR:gene1','HR_adj_p:gene1',
            'logFC:gene2','logFC_adj_p:gene2','HR:gene2','HR_adj_p:gene2',
            'logFC:gene3','logFC_adj_p:gene3','HR:gene3','HR_adj_p:gene3',
            'logFC:gene4','logFC_adj_p:gene4','HR:gene4','HR_adj_p:gene4',
            'logFC_Label:gene1','logFC_adj_p<0.05:gene1','logFC_Label:gene2','logFC_adj_p<0.05:gene2','logFC_Label:gene3','logFC_adj_p<0.05:gene3','logFC_Label:gene4','logFC_adj_p<0.05:gene4',
            'HR_Label:gene1','HR_adj_p<0.05:gene1','HR_Label:gene2','HR_adj_p<0.05:gene2','HR_Label:gene3','HR_adj_p<0.05:gene3','HR_Label:gene4','HR_adj_p<0.05:gene4',
            'Total_Combination_logFC','Total_Combination_HR'
        ]]

        print(subgraph_data)

        # Save the processed dataframe to excel file
        subgraph_data.to_excel(writer, sheet_name=subgraph_name, index=False)
# Print the path of the created Excel file
print(f"Excel file created at: {output_file_path}")






In [ ]:

#Check LogFC and HR that cannot find in TMMGDB


# List of columns to check for non-empty values
columns_to_check = [
    'logFC:gene1', 'logFC_adj_p:gene1', 'HR:gene1', 'HR_adj_p:gene1',
    'logFC:gene2', 'logFC_adj_p:gene2', 'HR:gene2', 'HR_adj_p:gene2',
    'logFC:gene3', 'logFC_adj_p:gene3', 'HR:gene3', 'HR_adj_p:gene3',
    'logFC:gene4', 'logFC_adj_p:gene4', 'HR:gene4', 'HR_adj_p:gene4'
]

# Load the Excel file
excel_file = pd.ExcelFile(output_file_path)

# Dictionary to store filtered data from each sheet
filtered_data = {}
consolidated_df = pd.DataFrame()  # DataFrame to store merged data

# Iterate through each sheet
for sheet_name in excel_file.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    # Check if all specified columns are present in the sheet
    if all(col in df.columns for col in columns_to_check):
        # Filter rows where all columns are non-empty
        filtered_df = df.dropna(subset=columns_to_check)
        
        # Store the filtered data
        filtered_data[sheet_name] = filtered_df

        # Print the sheet name and the filtered data
        print(f"\nFiltered data for sheet: {sheet_name}")
        if filtered_df.empty:
            print("No rows found with all non-empty values.")
        else:
            print(filtered_df)

        # Append to the consolidated DataFrame
        consolidated_df = pd.concat([consolidated_df, filtered_df], ignore_index=True)


# Output filtered data or save to a new Excel file
output_filtered_file = f"{outputfolder}/filtered_LogFC_HR_result_Pvalue-HR.xlsx"

with pd.ExcelWriter(output_filtered_file) as writer:
    for sheet_name, filtered_df in filtered_data.items():
        # Save each sheet to the new Excel file
        filtered_df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save the consolidated data as the last sheet
    consolidated_df.to_excel(writer, sheet_name="Consolidated", index=False)


print("\nFiltered data saved to 'filtered_LogFC_HR_result_Pvalue-HR.xlsx'.")

# Module02: Count the occurences of KIRC genes in 3 databases
### Step01: Convert GeneID to GeneSymbol

Download database
- The original database: https://www.genenames.org/, contains the source for approved human gene nomenclature
- The database is located at this site: https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/
- Downloading process takes around 1 minutes, if it takes more time than I suggest, please check for the other errors.
- If we fail download the file, we can use human_nomenclature_gene.txt that we already downloaded before.

In [ ]:
import requests

# URL of the file to download
url = "https://ftp.ebi.ac.uk/pub/databases/genenames/hgnc/tsv/non_alt_loci_set.txt"

# Define the destination directory
destination_directory = wkdr

# Send a GET request to the URL to download the file
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the destination file in binary write mode and write the downloaded content
    with open(f"{destination_directory}/human_nomenclature_gene.txt", "wb") as file:
        file.write(response.content)
    print("File downloaded successfully.")
else:
    print("Failed to download the file.")


**Handle the human nomenclature database**

In [ ]:
# Read in the dataframe

# Define the destination directory
destination_directory = wkdr

human_nomenclature_gene = pd.read_csv(f"{destination_directory}/human_nomenclature_gene.txt",sep = "\t")

# Taking only 2 columns into consideration: symbol and entrez_id
human_nomenclature_gene = human_nomenclature_gene[["symbol","entrez_id"]]

# Replace the Null value by 0
human_nomenclature_gene['entrez_id'] = human_nomenclature_gene['entrez_id'].fillna(0)

# Convert the type of values in "entrez_id" columns from float64 to int
human_nomenclature_gene["entrez_id"] = human_nomenclature_gene["entrez_id"].astype(int)
print(human_nomenclature_gene)

# Convert the type of human_nomenclature_gene: From dataframe to Dictionary
nomenclature_gene_dict = human_nomenclature_gene.set_index('entrez_id')['symbol'].to_dict()

In [ ]:
import os
import pandas as pd

# Define the working directory

# Get a list of all files in the folder DB
databases = f"{wkdr}/DBs"
databases_files = [os.path.join(databases, f) for f in os.listdir(databases) if os.path.isfile(os.path.join(databases, f))]

# Initialize an empty DataFrame
total_DB = pd.DataFrame()

# Loop through each database file
for DB_file in databases_files:
    print(f"Reading file: {DB_file}")

    # Get the database name (file name without extension)
    database_name = os.path.basename(DB_file).replace(".txt", "")

    # Skip if database_name is "data-Over3GeneList"
    if database_name == "data-Over3GeneList":
        continue  # Skip this file

    # Read the database file into a DataFrame
    df = pd.read_csv(DB_file, header=None)

    # Rename columns
    df.rename(columns={0: "GenID"}, inplace=True)

    # Add a new column with value 1 for this database
    df[database_name] = 1  

    # Set "GenID" as the index
    df.set_index("GenID", inplace=True)

    # Merge databases by adding new columns
    total_DB = pd.concat([total_DB, df], axis=1)  # Merge by columns

# Fill missing values with 0
total_DB.fillna(0, inplace=True)

# Map nomenclature gene names
total_DB["GenSymbol"] = total_DB.index.map(nomenclature_gene_dict)

# Ensure that the required columns exist before filtering
required_columns = ["GenSymbol", "CMGene", "HCMDB", "TMMGdb_3200"]
for col in required_columns:
    if col not in total_DB.columns:
        total_DB[col] = 0  # Add missing columns with default value 0

# Keep only relevant columns
total_DB = total_DB[required_columns]

# Calculate occurrences by summing across selected columns
total_DB["Occurrences"] = total_DB[['CMGene', 'HCMDB', 'TMMGdb_3200']].sum(axis=1)

# Print the final DataFrame
print(total_DB)

# Convert to dictionary with "GenSymbol" as key and "Occurrences" as value
total_DB_dict = total_DB.set_index('GenSymbol')['Occurrences'].to_dict()
print(total_DB_dict)

# Convert the dictionary to a DataFrame
df = pd.DataFrame(list(total_DB_dict.items()), columns=['Sheet Name', 'Order'])

# Export the DataFrame to a CSV file
df.to_csv(f"{wkdr}/output files/total_OCCinThreeDB_dict.csv", index=False)

# Print the DataFrame to verify
print(df)

### Step02: Count the occurrences of KIRC genes in databases
**Read-in the excel file**

In [ ]:
path_to_the_KIRC_gene = f"{wkdr}/output files/result_Pvalue-HR.xlsx"
path_to_the_KIRC_gene

**Handle one-by-one sheet**

In [ ]:
with pd.ExcelWriter(f"{wkdr}/output files/result_other_databases.xlsx") as writer:
    analyzed_subgraph = pd.read_excel(path_to_the_KIRC_gene, sheet_name=None)
    print(analyzed_subgraph)

    for sheet_name, processing_sheet in analyzed_subgraph.items():
        # Process each sheet as needed
        print(f"Sheet Name: {sheet_name}")

        # Map the gene in the network to total_DB_dict in order to get the occurences of this gene in 3 databases
        processing_sheet["Occurences:gene1"] = processing_sheet["gene1"].map(total_DB_dict)
        processing_sheet["Occurences:gene2"] = processing_sheet["gene2"].map(total_DB_dict)
        processing_sheet["Occurences:gene3"] = processing_sheet["gene3"].map(total_DB_dict)
        processing_sheet["Occurences:gene4"] = processing_sheet["gene4"].map(total_DB_dict)

        #Fill NA will O
        processing_sheet.fillna(0,inplace=True)

        #Sum the occurences of 3 genes in 3 database
        processing_sheet["Total_Occurences"] = processing_sheet[["Occurences:gene1","Occurences:gene2","Occurences:gene3","Occurences:gene4"]].sum(axis=1)

        processing_sheet.to_excel(writer,sheet_name = sheet_name,index=False)


# MODULE03: CANCER DRIVER GENES THE COSMIS DATABASE
**Description:**
Each subgraph contains 3 genes, the question is how many within 3 genes are presented in COSMIC database

Tier01: https://cancer.sanger.ac.uk/cosmic/census?tier=1

### Step01: Read the Cosmic database (Tier01)

In [ ]:
cosmic = pd.read_csv(f"{wkdr}/Census_Tier1.tsv",sep="\t")

cosmic["Cosmic_Tier1"] = 1

cosmic.rename(columns={"Gene Symbol":"GenSymbol"},inplace=True)
cosmic = cosmic[["GenSymbol","Cosmic_Tier1"]]

cosmic = cosmic.set_index('GenSymbol')['Cosmic_Tier1'].to_dict()
print(cosmic)

### Step03: Count the occurrence of genes in each subgraph in COSMIC database

In [ ]:
Occurrence_sheet = f"{wkdr}/output files/result_other_databases.xlsx"
with pd.ExcelWriter(f"{wkdr}/output files/result_Cosmic.xlsx") as writer:

    analyzed_subgraph = pd.read_excel(Occurrence_sheet, sheet_name=None)

    for sheet_name, processing_sheet in analyzed_subgraph.items():
        print(sheet_name)
        #print(processing_sheet)
        # Map the gene in the network to total_DB_dict in order to get the occurences of this gene in 3 databases
        processing_sheet["Cosmic:gene1"] = processing_sheet["gene1"].map(cosmic)
        processing_sheet["Cosmic:gene2"] = processing_sheet["gene2"].map(cosmic)
        processing_sheet["Cosmic:gene3"] = processing_sheet["gene3"].map(cosmic)
        processing_sheet["Cosmic:gene4"] = processing_sheet["gene4"].map(cosmic)
        
        processing_sheet = processing_sheet.rename(columns={"0":"index"})

        #Fill NA with 0
        processing_sheet.fillna(0,inplace=True)

        #Sum the occurences of 4 genes in 3 database
        processing_sheet["Total_Cosmic"] = processing_sheet[["Cosmic:gene1","Cosmic:gene2","Cosmic:gene3","Cosmic:gene4"]].sum(axis=1)

        processing_sheet.to_excel(writer,sheet_name = sheet_name,index=False)

In [ ]:
print(processing_sheet)

In [ ]:
pwd

# MODULE04: CFA calculation Avg and Weigthed Score Combination (using 11 combinations of the features)

In [ ]:
def re_normalize_column(x):
    re_value = (x - min(x)) / (max(x) - min(x))
    return re_value;

In [ ]:
import pandas as pd
import numpy as np


def rank_score_combination(input_sheet, output_file):
    # Define the desired order of sheets
    file_path = f"{wkdr}/output files/result_Cosmic.xlsx"
    xls = pd.ExcelFile(file_path)

# Create a dictionary mapping sheet names to their order
    sheet_order = {sheet_name: index for index, sheet_name in enumerate(xls.sheet_names)}

# Print the sheet order dictionary
    print(sheet_order)
    
    # Initialize an empty list to store data from all sheets
    combined_data = []
    
    # Read the sheet:
    analyzed_subgraph = pd.read_excel(input_sheet, sheet_name=None)
    
    # Sort the sheets by the desired order and combine data
    for sheet_name in sheet_order:
        if sheet_name in analyzed_subgraph:
            print(f"The sheet_id {sheet_name} is in processing")
            processing_sheet = analyzed_subgraph[sheet_name]
            processing_sheet["Sheet_Name"] = sheet_name  # Add a column to identify the source sheet
            combined_data.append(processing_sheet)
    
    # Concatenate all the sheets into a single DataFrame
    combined_df = pd.concat(combined_data, ignore_index=True)
    
    # Step 1: Scores normalization
    combined_df["slogFC"] = combined_df["Total_Combination_logFC"] / ((10*4) + 4)
    combined_df["sHR"] = combined_df["Total_Combination_HR"] / ((10*4) + 4)
    combined_df["sOcc"] = combined_df["Total_Occurences"] / 12
    combined_df["sCDG"] = combined_df["Total_Cosmic"] / 4
    
    # Calculate score combination and rank
    combined_df["SC"] = combined_df[["slogFC", "sHR", "sOcc", "sCDG"]].sum(axis=1)
    combined_df["Final Rank"] = combined_df["SC"].rank(ascending=False, method='average')
    
    # Re-normalize the columns
    combined_df["slogFC'"] = re_normalize_column(combined_df["slogFC"])
    combined_df["sHR'"] = re_normalize_column(combined_df["sHR"])
    combined_df["sOcc'"] = re_normalize_column(combined_df["sOcc"])
    combined_df["sCDG'"] = re_normalize_column(combined_df["sCDG"])
    
    # Calculate re-normalized score combination and rank
    combined_df["SC'"] = combined_df[["slogFC'", "sHR'", "sOcc'", "sCDG'"]].sum(axis=1)
    combined_df["Final Rank'"] = combined_df["SC'"].rank(ascending=False, method='average')
    
    # Ranking the columns
    combined_df["rlogFC"] = combined_df["slogFC'"].rank(ascending=False, method='dense')
    combined_df["rHR"] = combined_df["sHR'"].rank(ascending=False, method='dense')
    combined_df["rOcc"] = combined_df["sOcc'"].rank(ascending=False, method='dense')
    combined_df["rCDG"] = combined_df["sCDG'"].rank(ascending=False, method='dense')
    
    # Keep track of the gene
    combined_df["flogFC"] = combined_df["slogFC'"].reset_index(drop=True).sort_values(ascending=False).values
    
    sorted_indices = np.argsort(-combined_df["slogFC'"])
    sorted_index_labels = combined_df["index"].iloc[sorted_indices].index + 1
    sorted_index_labels_with_d = ['d' + str(label) for label in sorted_index_labels]
    combined_df["D_logFC"] = sorted_index_labels_with_d
    
    combined_df["fHR"] = combined_df["sHR'"].reset_index(drop=True).sort_values(ascending=False).values
    sorted_indices = np.argsort(-combined_df["sHR'"])
    sorted_index_labels = combined_df["index"].iloc[sorted_indices].index + 1
    sorted_index_labels_with_d = ['d' + str(label) for label in sorted_index_labels]
    combined_df["D_HR"] = sorted_index_labels_with_d
    
    combined_df["fOcc"] = combined_df["sOcc'"].reset_index(drop=True).sort_values(ascending=False).values
    sorted_indices = np.argsort(-combined_df["sOcc'"])
    sorted_index_labels = combined_df["index"].iloc[sorted_indices].index + 1
    sorted_index_labels_with_d = ['d' + str(label) for label in sorted_index_labels]
    combined_df["D_Occ"] = sorted_index_labels_with_d
    
    combined_df["fCDG"] = combined_df["sCDG'"].reset_index(drop=True).sort_values(ascending=False).values
    sorted_indices = np.argsort(-combined_df["sCDG'"])
    sorted_index_labels = combined_df["index"].iloc[sorted_indices].index + 1
    sorted_index_labels_with_d = ['d' + str(label) for label in sorted_index_labels]
    combined_df["D_CDG"] = sorted_index_labels_with_d
    
    # Save the combined results to a new Excel file
    combined_df.to_excel(output_file, sheet_name="Combined_Results", index=False)
    
# Example usage:
input_data = f"{wkdr}/output files/result_Cosmic.xlsx"
output_data = f"{wkdr}/output files/combined-rank-score-combination.xlsx"
rank_score_combination(input_data, output_data)


In [ ]:

#Check LogFC,HR,OCC,CDG that cannot find in TMMGDB


# List of columns to check for non-empty values
columns_to_check = [
    'logFC:gene1', 'logFC_adj_p:gene1', 'HR:gene1', 'HR_adj_p:gene1',
    'logFC:gene2', 'logFC_adj_p:gene2', 'HR:gene2', 'HR_adj_p:gene2',
    'logFC:gene3', 'logFC_adj_p:gene3', 'HR:gene3', 'HR_adj_p:gene3',
    'logFC:gene4', 'logFC_adj_p:gene4', 'HR:gene4', 'HR_adj_p:gene4'
]

# Load the Excel file
excel_file = pd.ExcelFile(output_data)

# Dictionary to store filtered data from each sheet
filtered_data = {}
consolidated_df = pd.DataFrame()  # DataFrame to store merged data

# Iterate through each sheet
for sheet_name in excel_file.sheet_names:
    # Read the sheet into a DataFrame
    df = pd.read_excel(excel_file, sheet_name=sheet_name)
    
    # Check if all specified columns are present in the sheet
    if all(col in df.columns for col in columns_to_check):
        # Filter rows where all columns are non-empty
        filtered_df = df[(df[columns_to_check] != 0).all(axis=1)]
        
        # Store the filtered data
        filtered_data[sheet_name] = filtered_df

        # Print the sheet name and the filtered data
        print(f"\nFiltered data for sheet: {sheet_name}")
        if filtered_df.empty:
            print("No rows found with all non-empty values.")
        else:
            print(filtered_df)

        # Append to the consolidated DataFrame
        consolidated_df = pd.concat([consolidated_df, filtered_df], ignore_index=True)


# Output filtered data or save to a new Excel file
output_filtered_file = f"{outputfolder}/filtered_LogFC_HR_OCC_CDG.xlsx"

with pd.ExcelWriter(output_filtered_file) as writer:
    for sheet_name, filtered_df in filtered_data.items():
        # Save each sheet to the new Excel file
        filtered_df.to_excel(writer, sheet_name=sheet_name, index=False)

    # Save the consolidated data as the last sheet
    consolidated_df.to_excel(writer, sheet_name="Consolidated", index=False)


print("\nFiltered data saved to 'filtered_LogFC_HR_OCC_CDG.xlsx'.")

In [ ]:
pip install xlsxwriter

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os


# Input file path (the Excel file with one sheet)
input_data = f"{wkdr}/output files/combined-rank-score-combination.xlsx"
output_directory = f"{wkdr}/output files"

# Read the single sheet from the Excel file
processing_sheet = pd.read_excel(output_data)

# Extract data for plotting
x_ranking = processing_sheet.index + 1  # Ranking is based on index + 1

y_Rank_SC_logFC = processing_sheet["flogFC"]
y_Rank_SC_HR = processing_sheet["fHR"]
y_Rank_SC_Occurences = processing_sheet["fOcc"]
y_Rank_SC_Cancer_Driver_Gene = processing_sheet["fCDG"]
# y_Rank_SC_PPIs = processing_sheet["fPPIs"]  # Uncomment if needed

# Calculate the maximum value of x-axis dynamically
max_x_value = len(processing_sheet)

# Create a plot for the data
plt.figure(figsize=(12, 6))
plt.plot(x_ranking, y_Rank_SC_logFC, color='blue', alpha=0.5, label='logFC', marker='o')
plt.plot(x_ranking, y_Rank_SC_HR, color='green', alpha=0.5, label='HR', marker='s')
plt.plot(x_ranking, y_Rank_SC_Occurences, color='red', alpha=0.5, label='Occ', marker='^')
plt.plot(x_ranking, y_Rank_SC_Cancer_Driver_Gene, color='orange', alpha=0.5, label='CDG', marker='x')
# plt.plot(x_ranking, y_Rank_SC_PPIs, color='purple', alpha=0.5, label='PPIs', marker='d')  # Uncomment if needed

# Add title and labels
plt.title('Rank-Score Characteristic Graph for 4-node Combined Data (All Subgraph ID)')
plt.xlabel('Rank')
plt.ylabel('Score')
plt.legend()
plt.grid(True)

# Set the maximum value of x-axis
plt.xlim(right=max_x_value)

# Set the x-axis ticks to show every nth value and adjust the font size
step_size = 19  # Adjust this to increase/decrease the spacing between labels
plt.xticks(range(1, max_x_value + 1, step_size), fontsize=8)  # Adjust 'fontsize' as needed


# Save the plot to a file
output_file = os.path.join(output_directory, "combined_plot_RSC.jpg")
plt.savefig(output_file, format='jpg', dpi=300)

# Display the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import os
from itertools import combinations

# Define working directory and input/output paths
input_data = f"{wkdr}/output files/combined-rank-score-combination.xlsx"
output_directory = f"{wkdr}/output files"

# Read the Excel file
processing_sheet = pd.read_excel(input_data)

# List of features
features = ["flogFC", "fHR", "fOcc", "fCDG"]  # Include all available features

# Generate combinations
combinations_4_features = list(combinations(features, 4))
combinations_3_features = list(combinations(features, 3))
combinations_2_features = list(combinations(features, 2))

# Combine all combinations into one list
all_combinations = combinations_4_features + combinations_3_features + combinations_2_features

# Define a color map for features
feature_colors = {
    "flogFC": "blue",
    "fHR": "green",
    "fOcc": "red",
    "fCDG": "orange",
    "fPPIs": "purple"  # Add more feature-color mappings as needed
}

# Loop through each combination and generate plots
for idx, combination in enumerate(all_combinations, 1):
    # Extract data for the current combination
    x_ranking = processing_sheet.index + 1  # Ranking is based on index + 1
    y_values = [processing_sheet[feature] for feature in combination]
    
    # Create a plot for the current combination
    plt.figure(figsize=(14, 8))
    
    # Plot each feature in the combination
    for y, feature in zip(y_values, combination):
        plt.plot(
            x_ranking, y, 
            color=feature_colors.get(feature, "black"),  # Default to black if no color is specified
            alpha=0.5, 
            label=feature, 
            marker='o'  # You can customize markers per feature if needed
        )
    
    # Add title and labels
    plt.title(f"Rank-Score Characteristic Graph for Combination of ({', '.join(combination)}) Features", fontsize=16)
    plt.xlabel('Rank', fontsize=12)
    plt.ylabel('Score', fontsize=12)
    
    # Add legend and grid
    plt.legend(fontsize=10, loc='upper right', frameon=True)
    plt.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.6)
    
    # Set the x-axis limits and ticks
    max_x_value = len(processing_sheet)
    plt.xlim(left=1, right=max_x_value)
    step_size = 7  # Adjust this to increase/decrease spacing between x-axis labels
    plt.xticks(range(1, max_x_value + 1, step_size), fontsize=10, rotation=45)
    
    # Save the plot
    output_file = os.path.join(output_directory, f"combined_plot_RSC_{idx}.jpg")
    plt.savefig(output_file, format='jpg', dpi=300)
    
    # Display the plot (optional)
    plt.show()

### CD and DS calculation for 11 features combinations

In [ ]:
import pandas as pd
import numpy as np
from itertools import combinations

# Update paths and file names as required
input_ranking = f"{wkdr}/output files/combined-rank-score-combination.xlsx"
output_results = f"{wkdr}/output files/weighted_calculation_results_combined_allsubgraph.xlsx"

# Load data
rank_data = pd.read_excel(input_ranking, sheet_name=None)
combined_data = pd.concat(rank_data.values(), ignore_index=True)

# Initialize lists to store the results
m = combined_data.shape[0]
print(f"The total number of rows in combined data is: {m}")

# Extract feature values and scores
FC = combined_data["flogFC"]
HR = combined_data["fHR"]
occ = combined_data["fOcc"]
cdg = combined_data["fCDG"]

feat = 4

# Extract score values
s_FC = combined_data["slogFC'"]
s_HR = combined_data["sHR'"]
s_occ = combined_data["sOcc'"]
s_cdg = combined_data["sCDG'"]

# Calculate avg_sc
#combined_data['avg_sc'] = (s_FC + s_HR + s_occ + s_cdg) / feat

# Rank based on avg_sc
#combined_data['rank_avg_sc'] = combined_data['avg_sc'].rank(ascending=False)

# Print the scores
print("s_FC :", s_FC)

# Cognitive Diversity (CD) Calculation
CD_FC_HR = np.sqrt(np.sum((FC - HR) ** 2) / m)
CD_FC_occ = np.sqrt(np.sum((FC - occ) ** 2) / m)
CD_FC_cdg = np.sqrt(np.sum((FC - cdg) ** 2) / m)
CD_HR_occ = np.sqrt(np.sum((HR - occ) ** 2) / m)
CD_HR_cdg = np.sqrt(np.sum((HR - cdg) ** 2) / m)
CD_occ_cdg = np.sqrt(np.sum((occ - cdg) ** 2) / m)

def calculate_diversity_strength(n, features, CD_matrix):
    ds = {}
    for i in range(len(features)):
        for j in range(i + 1, len(features)):
            feature1 = features[i]
            feature2 = features[j]
            ds[feature1] = ds.get(feature1, 0) + CD_matrix.get((feature1, feature2), 0)
            ds[feature2] = ds.get(feature2, 0) + CD_matrix.get((feature1, feature2), 0)

    for key in ds:
        ds[key] = ds[key] * (1 / (n - 1))

    return ds

def calculate_weighted_ds(row, ds, s_scores, features):
    total_weighted_score = 0
    sum_ds = 0

    # Dynamically calculate the weighted score for the given features
    for feature in features:
        total_weighted_score += ds.get(feature, 0) * row[s_scores[feature]]
        sum_ds += ds.get(feature, 0)

    # Avoid division by zero
    if sum_ds == 0:
        return 0

    return total_weighted_score / sum_ds

# Define the feature set
features = ['FC', 'HR', 'occ', 'cdg']

# Define the Cognitive Diversity matrix for different feature pairs
CD_matrix = {
    ('FC', 'HR'): CD_FC_HR,
    ('FC', 'occ'): CD_FC_occ,
    ('FC', 'cdg'): CD_FC_cdg,
    ('HR', 'occ'): CD_HR_occ,
    ('HR', 'cdg'): CD_HR_cdg,
    ('occ', 'cdg'): CD_occ_cdg
}

# Define the s_scores (these represent the actual column values from combined_data)
s_scores = {
    'FC': "slogFC'",
    'HR': "sHR'",
    'occ': "sOcc'",
    'cdg': "sCDG'"
}

# Generate all combinations of 4, 3, and 2 features
combinations_4_features = list(combinations(features, 4))
combinations_3_features = list(combinations(features, 3))
combinations_2_features = list(combinations(features, 2))

# Initialize a dictionary to store the results
results_dict = {}

# Function to prepare the dataframe for each combination
def create_combination_df(combo, CD_matrix, combined_data):
    df = combined_data.copy()

    # Add CD values
    for i in range(len(combo)):
        for j in range(i + 1, len(combo)):
            pair = (combo[i], combo[j])
            column_name = f"CD_{combo[i]}_{combo[j]}"
            df[column_name] = CD_matrix.get(pair, 0)

    # Add DS values for each feature
    ds_result = calculate_diversity_strength(len(combo), combo, CD_matrix)
    for feature in combo:
        df[f"ds_{feature}"] = ds_result.get(feature, 0)

    # Calculate weighted diversity strength for each row using the correct formula
    df["weighted_ds"] = df.apply(lambda row: calculate_weighted_ds(row, ds_result, s_scores, combo), axis=1)

    # Rank the weighted_ds
    df["weighted_ds_rank"] = df["weighted_ds"].rank(ascending=False, method='average')


     # Calculate avg_sc based on the features in the current combination
    score_columns = [s_scores[feature] for feature in combo]
    df['avg_sc'] = df[score_columns].mean(axis=1)

    # Rank avg_sc based on the calculated avg_sc for each row
    df['rank_avg_sc'] = df['avg_sc'].rank(ascending=False, method='average')

    return df

# Process all feature combinations
for combo in combinations_4_features + combinations_3_features + combinations_2_features:
    sheet_name = "_".join(combo)
    combo_df = create_combination_df(combo, CD_matrix, combined_data)
    results_dict[sheet_name] = combo_df

# Write the results to an Excel file with separate sheets for each combination
with pd.ExcelWriter(output_results, engine='openpyxl') as writer:
    for sheet_name, df in results_dict.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

print("Excel file with all combinations, CD/DS values, avg_sc, and rankings has been saved.")

## Module05: Find TOP2 and BOTTOM2 from Weigthed Score Combination result

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Define the working directory for saving the plots
#wkdr = "/Users/anindaastuti/CFA 4node combine features/output files"

# Function to plot the result for a specific feature combination and save the figure
def plot_weighted_ds_vs_data(df, combination_name):
    # Create a plot
    plt.figure(figsize=(10, 6))

    # Add a column for data labels (d1, d2, ..., dn)
    df['data'] = [f'd{i+1}' for i in range(len(df))]

    # Sort the dataframe based on weighted diversity score rank to get top and bottom 2
    df_sorted = df.sort_values(by="weighted_ds_rank")

    # Get the top 2 and bottom 2 ranked rows
    top_2 = df_sorted.head(2)
    bottom_2 = df_sorted.tail(2)

    # Plot all the points using a default blue color
    plt.scatter(df['data'], df['weighted_ds'], color='blue', label='Other ranks')

    # Highlight the top 2 points with different colors
    top1_data = top_2.iloc[0]['data']
    top1_ds = top_2.iloc[0]['weighted_ds']
    top2_data = top_2.iloc[1]['data']
    top2_ds = top_2.iloc[1]['weighted_ds']
    
    plt.scatter(top1_data, top1_ds, color='red', label='Top 1 rank')
    plt.scatter(top2_data, top2_ds, color='orange', label='Top 2 rank')

    # Highlight the bottom 2 points with different colors
    bottom1_data = bottom_2.iloc[0]['data']
    bottom1_ds = bottom_2.iloc[0]['weighted_ds']
    bottom2_data = bottom_2.iloc[1]['data']
    bottom2_ds = bottom_2.iloc[1]['weighted_ds']
    
    plt.scatter(bottom1_data, bottom1_ds, color='brown', label='Bottom 1 rank')
    plt.scatter(bottom2_data, bottom2_ds, color='green', label='Bottom 2 rank')

    # Annotate the top 2 points with their data label
    plt.annotate(f'{top1_data}', (top1_data, top1_ds),
                 textcoords="offset points", xytext=(0, 5), ha='center', color='red', fontsize=7)
    plt.annotate(f'{top2_data}', (top2_data, top2_ds),
                 textcoords="offset points", xytext=(0, 5), ha='center', color='orange', fontsize=7)

    # Annotate the bottom 2 points with their data label
    plt.annotate(f'{bottom1_data}', (bottom1_data, bottom1_ds),
                 textcoords="offset points", xytext=(0, -10), ha='center', color='brown', fontsize=7)
    plt.annotate(f'{bottom2_data}', (bottom2_data, bottom2_ds),
                 textcoords="offset points", xytext=(0, -10), ha='center', color='green', fontsize=7)

    # Add labels and title
    plt.xlabel('Samples Data (d1 to d114)')
    plt.ylabel('Weighted_sc')
    plt.title(f'Weighted Score Combination for {combination_name} Features')

    # Customize x-axis labels with intervals of 5
    plt.xticks(ticks=range(0, len(df), 5), labels=[f'd{i+1}' for i in range(0, len(df), 5)], rotation=90)

    # Add a legend to explain the color scheme
    plt.legend()

    # Remove the grid background
    plt.grid(False)

    # Adjust the plot to fit in the top-right corner
    plt.xlim(left=0)  # Adjust as needed
    plt.ylim(bottom=0)  # Adjust as needed

    # Reduce padding to fit the plot area
    plt.subplots_adjust(right=0.95, top=0.95, left=0.05, bottom=0.05)

    # Save the plot with 300 DPI in the specified directory
    filename = f"{wkdr}/{combination_name}_plot.png"
    plt.savefig(filename, dpi=300, bbox_inches='tight')

    # Display the plot
    plt.tight_layout()
    plt.show()

# Process all feature combinations and plot the results
for combo in combinations_4_features + combinations_3_features + combinations_2_features:
    # Get the sheet name based on the combination of features
    sheet_name = "_".join(combo)

    # Extract the DataFrame for this combination from the results_dict
    combo_df = results_dict[sheet_name]

    # Call the plotting function for this specific combination
    plot_weighted_ds_vs_data(combo_df, sheet_name)

In [ ]:
import pandas as pd

# Define the new file path for saving the top 2 and bottom 2 results
new_excel_file = "/Users/anindaastuti/CFA 4node combine features/output files/top_bottom_ranks.xlsx"

# Function to process the top and bottom 2 ranks for a specific feature combination and save to Excel
def save_top_bottom_to_new_excel(df, combination_name, writer):
    # Add a column for data labels (d1, d2, ..., dn)
    df['data'] = [f'd{i+1}' for i in range(len(df))]

    # Sort the dataframe based on weighted diversity score rank to get top and bottom 2
    df_sorted = df.sort_values(by="weighted_ds_rank")

    # Get the top 2 and bottom 2 ranked rows
    top_2 = df_sorted.head(2)
    bottom_2 = df_sorted.tail(2)

    # Combine the top and bottom 2 into one dataframe
    top_bottom_df = pd.concat([top_2, bottom_2])

    # Write the top and bottom 2 data to the new Excel file
    top_bottom_df.to_excel(writer, sheet_name=f'Top_Bottom_{combination_name}', index=False)

# Create a new Excel file and save the top/bottom 2 data for all combinations
with pd.ExcelWriter(new_excel_file, engine='openpyxl') as writer:
    for combo in combinations_4_features + combinations_3_features + combinations_2_features:
        # Get the sheet name based on the combination of features
        sheet_name = "_".join(combo)

        # Extract the DataFrame for this combination from the results_dict
        combo_df = results_dict[sheet_name]

        # Call the function to save the top and bottom 2 for this combination to the new Excel file
        save_top_bottom_to_new_excel(combo_df, sheet_name, writer)

## Module06: Calculate frequency item-ID TOP5 and BOTTOM5.
It is chosen from the highest and lowest five ranked items-Id for each of the 11 combinations.

In [ ]:
#top 5 or 10 and bottom 5 0r 10 frequency
import pandas as pd
from collections import Counter

# Define the new file path for saving the top 5 and bottom 5 results and summary
new_excel_file = "/Users/anindaastuti/CFA 4node combine features/output files/summary_top_bottom_10_subgraph_ids.xlsx"

# Initialize dictionaries to store top and bottom 5 subgraph IDs for each combination
summary_top_dict = {}
summary_bottom_dict = {}
n_pick=10
# Function to process the top and bottom 5 ranks for a specific feature combination
def process_top_bottom_5_for_combination(df, combination_name):
    # Sort the dataframe based on weighted diversity score rank to get top and bottom 5
    df_sorted = df.sort_values(by="weighted_ds_rank")
    
    top_5 = df_sorted.head(n_pick)  # Get the top 5
    bottom_5 = df_sorted.tail(n_pick)  # Get the bottom 5

    # Collect subgraph IDs from the 'data' column for top and bottom
    top_subgraph_ids = top_5['data'].tolist()
    bottom_subgraph_ids = bottom_5['data'].tolist()

    # Store the top and bottom 5 subgraph IDs in the summary dictionaries
    summary_top_dict[combination_name] = top_subgraph_ids
    summary_bottom_dict[combination_name] = bottom_subgraph_ids

    # Return both lists for frequency analysis
    return top_subgraph_ids, bottom_subgraph_ids

# Create a new Excel file and save the top/bottom 5 data for all combinations
with pd.ExcelWriter(new_excel_file, engine='openpyxl') as writer:
    all_top_ids = []     # To store all top 5 IDs across all combinations
    all_bottom_ids = []  # To store all bottom 5 IDs across all combinations
    
    for combo in combinations_4_features + combinations_3_features + combinations_2_features:
        # Get the sheet name based on the combination of features
        sheet_name = "_".join(combo)

        # Extract the DataFrame for this combination from the results_dict
        combo_df = results_dict[sheet_name]

        # Process the top and bottom 5 and store them
        top_5_ids, bottom_5_ids = process_top_bottom_5_for_combination(combo_df, sheet_name)
        
        # Add the top and bottom 5 IDs to the global lists
        all_top_ids.extend(top_5_ids)
        all_bottom_ids.extend(bottom_5_ids)

    # Calculate frequency of each subgraph ID across all combinations for both top and bottom
    top_subgraph_count = Counter(all_top_ids)
    bottom_subgraph_count = Counter(all_bottom_ids)

    most_common_top_subgraphs = top_subgraph_count.most_common()
    most_common_bottom_subgraphs = bottom_subgraph_count.most_common()

    # Prepare summary DataFrames with the top and bottom 5 subgraph IDs for each combination
    summary_top_df = pd.DataFrame(summary_top_dict)
    summary_bottom_df = pd.DataFrame(summary_bottom_dict)

    summary_top_df.index = [f'Top {i+1}' for i in range(n_pick)]     # Label the rows as Top 1, Top 2, ..., Top 5
    summary_bottom_df.index = [f'Bottom {i+1}' for i in range(n_pick)]  # Label the rows as Bottom 1, Bottom 2, ..., Bottom 5

    # Save the summary tables with top 5 and bottom 5 subgraph IDs for each combination
    summary_top_df.to_excel(writer, sheet_name='Top_10_Subgraph_IDs_Summary')
    summary_bottom_df.to_excel(writer, sheet_name='Bottom_10_Subgraph_IDs_Summary')

    # Save the most common subgraph IDs across all combinations for both top and bottom
    common_top_subgraphs_df = pd.DataFrame(most_common_top_subgraphs, columns=['Top_Subgraph_ID', 'Frequency_Top'])
    common_bottom_subgraphs_df = pd.DataFrame(most_common_bottom_subgraphs, columns=['Bottom_Subgraph_ID', 'Frequency_Bottom'])

    common_top_subgraphs_df.to_excel(writer, sheet_name='Most_Common_Top_Subgraph_IDs', index=False)
    common_bottom_subgraphs_df.to_excel(writer, sheet_name='Most_Common_Bottom_Subgraph_IDs', index=False)

## Module07: Calculate Jaccard index to Quantify the difference between the weighted SC and average SC

In [ ]:
#find the top 2 and bottom 2 for Avg_Sc
import pandas as pd

# Define the new file path for saving the top 2 and bottom 2 results
new_excel_file = "/Users/anindaastuti/CFA 4node combine features/output files/Avg_Sc_top_bottom_ranks.xlsx"

# Function to process the top and bottom 2 ranks for a specific feature combination and save to Excel
def save_top_bottom_to_new_excel(df, combination_name, writer):
    # Add a column for data labels (d1, d2, ..., dn)
    df['data'] = [f'd{i+1}' for i in range(len(df))]

    # Sort the dataframe based on weighted diversity score rank to get top and bottom 2
    df_sorted = df.sort_values(by="rank_avg_sc")

    # Get the top 2 and bottom 2 ranked rows
    top_2 = df_sorted.head(2)
    bottom_2 = df_sorted.tail(2)

    # Combine the top and bottom 2 into one dataframe
    top_bottom_df = pd.concat([top_2, bottom_2])

    # Write the top and bottom 2 data to the new Excel file
    top_bottom_df.to_excel(writer, sheet_name=f'Top_Bottom_{combination_name}', index=False)

# Create a new Excel file and save the top/bottom 2 data for all combinations
with pd.ExcelWriter(new_excel_file, engine='openpyxl') as writer:
    for combo in combinations_4_features + combinations_3_features + combinations_2_features:
        # Get the sheet name based on the combination of features
        sheet_name = "_".join(combo)

        # Extract the DataFrame for this combination from the results_dict
        combo_df = results_dict[sheet_name]

        # Call the function to save the top and bottom 2 for this combination to the new Excel file
        save_top_bottom_to_new_excel(combo_df, sheet_name, writer)

In [ ]:
#top 5 and bottom 5 frequency
import pandas as pd
from collections import Counter

# Define the new file path for saving the top 5 and bottom 5 results and summary
new_excel_file = "/Users/anindaastuti/CFA 4node combine features/output files/avg_Sc_summary_top_bottom_10_subgraph_ids.xlsx"

# Initialize dictionaries to store top and bottom 5 subgraph IDs for each combination
summary_top_dict = {}
summary_bottom_dict = {}

# Function to process the top and bottom 5 ranks for a specific feature combination
def process_top_bottom_5_for_combination(df, combination_name):
    # Sort the dataframe based on weighted diversity score rank to get top and bottom 5
    df_sorted = df.sort_values(by="rank_avg_sc")
    
    top_5 = df_sorted.head(n_pick)  # Get the top 5
    bottom_5 = df_sorted.tail(n_pick)  # Get the bottom 5

    # Collect subgraph IDs from the 'data' column for top and bottom
    top_subgraph_ids = top_5['data'].tolist()
    bottom_subgraph_ids = bottom_5['data'].tolist()

    # Store the top and bottom 5 subgraph IDs in the summary dictionaries
    summary_top_dict[combination_name] = top_subgraph_ids
    summary_bottom_dict[combination_name] = bottom_subgraph_ids

    # Return both lists for frequency analysis
    return top_subgraph_ids, bottom_subgraph_ids

# Create a new Excel file and save the top/bottom 5 data for all combinations
with pd.ExcelWriter(new_excel_file, engine='openpyxl') as writer:
    all_top_ids = []     # To store all top 5 IDs across all combinations
    all_bottom_ids = []  # To store all bottom 5 IDs across all combinations
    
    for combo in combinations_4_features + combinations_3_features + combinations_2_features:
        # Get the sheet name based on the combination of features
        sheet_name = "_".join(combo)

        # Extract the DataFrame for this combination from the results_dict
        combo_df = results_dict[sheet_name]

        # Process the top and bottom 5 and store them
        top_5_ids, bottom_5_ids = process_top_bottom_5_for_combination(combo_df, sheet_name)
        
        # Add the top and bottom 5 IDs to the global lists
        all_top_ids.extend(top_5_ids)
        all_bottom_ids.extend(bottom_5_ids)

    # Calculate frequency of each subgraph ID across all combinations for both top and bottom
    top_subgraph_count = Counter(all_top_ids)
    bottom_subgraph_count = Counter(all_bottom_ids)

    most_common_top_subgraphs = top_subgraph_count.most_common()
    most_common_bottom_subgraphs = bottom_subgraph_count.most_common()

    # Prepare summary DataFrames with the top and bottom 5 subgraph IDs for each combination
    summary_top_df = pd.DataFrame(summary_top_dict)
    summary_bottom_df = pd.DataFrame(summary_bottom_dict)

    summary_top_df.index = [f'Top {i+1}' for i in range(n_pick)]     # Label the rows as Top 1, Top 2, ..., Top 5
    summary_bottom_df.index = [f'Bottom {i+1}' for i in range(n_pick)]  # Label the rows as Bottom 1, Bottom 2, ..., Bottom 5

    # Save the summary tables with top 5 and bottom 5 subgraph IDs for each combination
    summary_top_df.to_excel(writer, sheet_name='Top_10_Subgraph_IDs_Summary')
    summary_bottom_df.to_excel(writer, sheet_name='Bottom_10_Subgraph_IDs_Summary')

    # Save the most common subgraph IDs across all combinations for both top and bottom
    common_top_subgraphs_df = pd.DataFrame(most_common_top_subgraphs, columns=['Top_Subgraph_ID', 'Frequency_Top'])
    common_bottom_subgraphs_df = pd.DataFrame(most_common_bottom_subgraphs, columns=['Bottom_Subgraph_ID', 'Frequency_Bottom'])

    common_top_subgraphs_df.to_excel(writer, sheet_name='Most_Common_Top_Subgraph_IDs', index=False)
    common_bottom_subgraphs_df.to_excel(writer, sheet_name='Most_Common_Bottom_Subgraph_IDs', index=False)

In [ ]:
import pandas as pd

# Load the Excel files
file1 = f"{wkdr}/avg_Sc_summary_top_bottom_10_subgraph_ids.xlsx"
file2 = f"{wkdr}/summary_top_bottom_10_subgraph_ids.xlsx"

# Specify the sheet names to compare
sheets_to_compare = ["Top_10_Subgraph_IDs_Summary", "Bottom_10_Subgraph_IDs_Summary"]

# List of columns to compare (assuming these are the 4-feature combination columns)
columns_to_compare = [
    'FC_HR_occ_cdg', 'FC_HR_occ', 'FC_HR_cdg', 'FC_occ_cdg', 
    'HR_occ_cdg', 'FC_HR', 'FC_occ', 'FC_cdg', 'HR_occ', 'HR_cdg', 'occ_cdg'
]

# Function to calculate the Jaccard index between two sets
def jaccard_index(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Dictionary to store results
jaccard_results = {}

# Loop through each sheet and calculate the Jaccard index for each column
for sheet in sheets_to_compare:
    # Load each sheet from both files
    df1 = pd.read_excel(file1, sheet_name=sheet)
    df2 = pd.read_excel(file2, sheet_name=sheet)
    
    # Calculate Jaccard index for each specified column in the sheet
    sheet_results = {}
    for col in columns_to_compare:
        set1 = set(df1[col].dropna())  # Convert to set and remove NaN values
        set2 = set(df2[col].dropna())  # Convert to set and remove NaN values
        sheet_results[col] = jaccard_index(set1, set2)
    
    # Store results for the current sheet
    jaccard_results[sheet] = sheet_results

# Convert results to a DataFrame
result_df = pd.DataFrame(jaccard_results).reset_index()
result_df.columns = ['Column', 'Jaccard_Index_Top_10', 'Jaccard_Index_Bottom_10']

# Save the DataFrame to an Excel file
output_file = "/Users/anindaastuti/CFA 4node combine features/output files/jaccard_index_results_by_sheet_top_bott_10.xlsx"
result_df.to_excel(output_file, index=False)

print(f"Jaccard index results saved to {output_file}")

### THIS IS THE END OF THE PROGRAM, PLEASE CHECK THE FINAL RESULT FILE in output files folder